In [1]:
import torch.nn as nn
import numpy as np
import torch
import librosa
import os

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import KFold

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using CUDA GPU for this project

In [2]:
num_labels = 24

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(device)

cuda:0


Here I have created some functions that we'll be using for our processing and preparation phase

In [3]:
from skimage.transform import resize
from skimage.filters import gaussian
from skimage.color import rgb2gray
from skimage import exposure, util


def horizontal_flip(img):
    horizontal_flip_img = img[:, ::-1]
    return addChannels(horizontal_flip_img)

def vertical_flip(img):
    vertical_flip_img = img[::-1, :]
    return addChannels(vertical_flip_img)

def addNoisy(img):
    noise_img = util.random_noise(img)
    return addChannels(noise_img)

def contrast_stretching(img):
    contrast_img = exposure.rescale_intensity(img)
    return addChannels(contrast_img)

def randomGaussian(img):
    gaussian_img = gaussian(img)
    return addChannels(gaussian_img)

def grayScale(img):
    gray_img = rgb2gray(img)
    return addChannels(gray_img)

def randomGamma(img):
    img_gamma = exposure.adjust_gamma(img)
    return addChannels(img_gamma)

def addChannels(img):
    return np.stack((img, img, img))

def spec_to_image(spec):
    spec = resize(spec, (224, 400))
    eps=1e-6
    mean = spec.mean()
    std = spec.std()
    spec_norm = (spec - mean) / (std + eps)
    spec_min, spec_max = spec_norm.min(), spec_norm.max()
    spec_scaled = 255 * (spec_norm - spec_min) / (spec_max - spec_min)
    spec_scaled = spec_scaled.astype(np.uint8)
    spec_scaled = np.asarray(spec_scaled)
    return spec_scaled

In [4]:
import pandas as pd

sr = 48000
length = 10 * sr
data = pd.read_csv("../input/rfcx-species-audio-detection/train_tp.csv")

fmin = sr / 2
fmax = 0
for i in range(0, len(data)):
    if fmin > float(data.iloc[i]['f_min']):
        fmin = float(data.iloc[i]['f_min'])
    if fmax < float(data.iloc[i]['f_max']):
        fmax = float(data.iloc[i]['f_max'])
        
fmin = int(fmin * 0.9)
fmax = int(fmax * 1.1)

label_list = []
data_list = []
audio_data = {}
for i in range(0, len(data)):
    recording_id = data.recording_id.values[i]
    species_id = int(data.species_id.values[i])
    data_list.append(recording_id)
    label_list.append(species_id)

    wav, sr = librosa.load('../input/rfcx-species-audio-detection/train/' + recording_id + '.flac', sr=None)
    t_min = float(data.t_min.values[i]) * sr
    t_max = float(data.t_max.values[i]) * sr
    center = np.round((t_min + t_max) / 2)
    beginning = center - length / 2
    if beginning < 0:
        beginning = 0
    ending = beginning + length
    if ending > len(wav):
        ending = len(wav)
        beginning = ending - length
    slice = wav[int(beginning):int(ending)]
    
    spec=librosa.feature.melspectrogram(y=slice, sr=sr, fmin=fmin, fmax=fmax)
    spec_db=librosa.power_to_db(spec, top_db=80)
    
    img = spec_to_image(spec_db)
    
    audio_data[recording_id] = img

In [5]:
import copy
from tqdm import tqdm

learning_rate = 1e-4
epochs = 20
loss_fn = nn.CrossEntropyLoss()

def train(model, loss_fn, train_loader, valid_loader, epochs, optimizer, scheduler):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    train_losses = []
    valid_losses = []
    
    for epoch in tqdm(range(1,epochs+1)):
        model.train()
        batch_losses=[]
        for _, data in enumerate(train_loader):
            x, y = data
            optimizer.zero_grad()
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.long)
            y_hat = model(x)
            loss = loss_fn(y_hat, y)
            loss.backward()
            batch_losses.append(loss.item())
            optimizer.step()
        train_losses.append(batch_losses)

        model.eval()
        batch_losses=[]
        trace_y = []
        trace_yhat = []
        
        for _, data in enumerate(valid_loader):
            x, y = data
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.long)
            y_hat = model(x)
            loss = loss_fn(y_hat, y)
            trace_y.append(y.cpu().detach().numpy())
            trace_yhat.append(y_hat.cpu().detach().numpy())      
            batch_losses.append(loss.item())
        valid_losses.append(batch_losses)
        trace_y = np.concatenate(trace_y)
        trace_yhat = np.concatenate(trace_yhat)
        accuracy = np.mean(trace_yhat.argmax(axis=1)==trace_y)
        
        print("epoch = %d, train_loss = %.5f, val_loss = %.5f, val_accuracy = %.5f" % (epoch, np.mean(train_losses[-1]), np.mean(valid_losses[-1]), accuracy))

        scheduler.step(np.mean(valid_losses[-1]))
        if accuracy > best_acc:
            best_acc = accuracy
            best_model_wts = copy.deepcopy(model.state_dict())

    model.load_state_dict(best_model_wts)
    return model

In [6]:
import random

class AudioData(Dataset):
    def __init__(self, X, y, data_type):
        self.data = []
        self.labels = []
        self.augs = [addNoisy, contrast_stretching,randomGaussian,randomGamma, vertical_flip, horizontal_flip, addChannels]
        self.data_type=data_type
        for i in range(0, len(X)):
            recording_id = X[i]
            label = y[i]
            mel_spec = audio_data[recording_id]
            self.data.append(mel_spec)
            self.labels.append(label)
                
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if self.data_type == "train":
            aug= random.choice(self.augs)
            data = aug(self.data[idx])
        else:
            data = addChannels(self.data[idx])
        return data, self.labels[idx]

In [7]:
from torchvision.models import resnet101

def get_model():
    model = resnet101(pretrained=True)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, num_labels)
    model = model.to(device)
    return model

In [8]:
fold_num = 4
skf = KFold(n_splits=fold_num, shuffle=True, random_state=32)

for fold_id, (train_index, val_index) in enumerate(skf.split(data_list, label_list)):
    X_train = np.take(data_list, train_index)
    y_train = np.take(label_list, train_index, axis = 0)
    X_val = np.take(data_list, val_index)
    y_val = np.take(label_list, val_index, axis = 0)

    train_data = AudioData(X_train, y_train, "train")
    valid_data = AudioData(X_val, y_val, "valid")
    train_loader = DataLoader(train_data, batch_size=8, shuffle=True, drop_last=True)
    valid_loader = DataLoader(valid_data, batch_size=8, shuffle=True, drop_last=True)

    model = get_model()
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3)
    model = train(model, loss_fn, train_loader, valid_loader, epochs, optimizer, scheduler)
    torch.save(model.state_dict(), "./model" + str(fold_id) + ".pt")
    
    del train_data, valid_data, train_loader, valid_loader, model, X_train, X_val, y_train, y_val

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:00<00:00, 320MB/s]
  5%|▌         | 1/20 [00:25<08:03, 25.43s/it]

epoch = 1, train_loss = 2.92614, val_loss = 2.47244, val_accuracy = 0.28289


 10%|█         | 2/20 [00:45<06:38, 22.12s/it]

epoch = 2, train_loss = 2.40460, val_loss = 2.44471, val_accuracy = 0.35855


 15%|█▌        | 3/20 [01:05<05:59, 21.17s/it]

...

epoch = 18, train_loss = 0.45195, val_loss = 1.12117, val_accuracy = 0.78289


 95%|█████████▌| 19/20 [06:16<00:19, 19.79s/it]

epoch = 19, train_loss = 0.32024, val_loss = 1.11997, val_accuracy = 0.80921


100%|██████████| 20/20 [06:35<00:00, 19.80s/it]

epoch = 20, train_loss = 0.22147, val_loss = 1.06084, val_accuracy = 0.80592


In [9]:
def load_test_file(f):
    wav, sr = librosa.load('../input/rfcx-species-audio-detection/test/' + f, sr=None)

    segments = len(wav) / length
    segments = int(np.ceil(segments))
    
    mel_array = []
    
    for i in range(0, segments):
        if (i + 1) * length > len(wav):
            slice = wav[len(wav) - length:len(wav)]
        else:
            slice = wav[i * length:(i + 1) * length]
        
        spec=librosa.feature.melspectrogram(y=slice, sr=sr, fmin=fmin, fmax=fmax)
        spec_db=librosa.power_to_db(spec,top_db=80)

        img = spec_to_image(spec_db)
        mel_spec = np.stack((img, img, img))
        mel_array.append(mel_spec)
    
    return mel_array

In [10]:
members = []
for i in range(fold_num):
    model = get_model()
    model.load_state_dict(torch.load('./model'+str(i)+'.pt'))
    model.eval()
    members.append(model)
    
os.remove('./model0.pt') 
os.remove('./model1.pt')
os.remove('./model2.pt') 
os.remove('./model3.pt')

In [11]:
import csv

with open('submission.csv', 'w', newline='') as csvfile:
    submission_writer = csv.writer(csvfile, delimiter=',')
    submission_writer.writerow(['recording_id','s0','s1','s2','s3','s4','s5','s6','s7','s8','s9','s10','s11',
                               's12','s13','s14','s15','s16','s17','s18','s19','s20','s21','s22','s23'])
    
    test_files = os.listdir('../input/rfcx-species-audio-detection/test/')
    print(len(test_files))
    
    for i in range(0, len(test_files)):
        data = load_test_file(test_files[i])
        data = torch.tensor(data)
        data = data.float()
        if torch.cuda.is_available():
            data = data.cuda()

        output_list = []
        for m in members:
            output = m(data)
            maxed_output = torch.max(output, dim=0)[0]
            maxed_output = maxed_output.cpu().detach()
            output_list.append(maxed_output)
        avg_maxed_output = torch.mean(torch.stack(output_list), dim=0)
        
        file_id = str.split(test_files[i], '.')[0]
        write_array = [file_id]
        
        for out in avg_maxed_output:
            write_array.append(out.item())
    
        submission_writer.writerow(write_array)


1992


/tmp/ipykernel_26/1623507928.py:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:245.)
  data = torch.tensor(data)
